In [1]:
import open3d as o3d
import numpy as np
import os

# Directory where the file is located
directory = "D:/COMPLEX_YOLO/VOD/view_of_delft_PUBLIC/lidar/training/velodyne"

# Specify the file you want to process
filename = "00544.bin"

# Load binary point cloud
bin_pcd = np.fromfile(os.path.join(directory, filename), dtype=np.float32)
points = bin_pcd.reshape((-1, 4))[:, 0:3]

# Convert to Open3D point cloud
o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))

# Filter points 
filtered_points = [point for point in o3d_pcd.points if point[2] < 2.0 and point[1] < 20.0 and point[1] > -20.0 and point[0] < 100 and point[0] > 0.0]

# Convert to Open3D point cloud
filtred_PCL = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(filtered_points))

# Visualize the filtered point cloud
o3d.visualization.draw_geometries([filtred_PCL])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def ransac(point_cloud, distance_threshold=0.1, ransac_n=3, num_iterations=2000):
    """
    RANSAC-based plane segmentation for a point cloud.
    Parameters:
        point_cloud (open3d.geometry.PointCloud): The input point cloud.
        distance_threshold (float, optional): The maximum distance a point can be from the plane to be considered an inlier.
            Default is 0.33.
        ransac_n (int, optional): The number of points to randomly sample for each iteration of RANSAC. Default is 3.
        num_iterations (int, optional): The number of RANSAC iterations to perform. Default is 100.
    Returns:
        open3d.geometry.PointCloud, open3d.geometry.PointCloud: Two point clouds representing the inliers and outliers
        of the segmented plane, respectively.
    """
    # Perform plane segmentation using RANSAC
    plane_model, inliers = point_cloud.segment_plane(distance_threshold=distance_threshold, ransac_n=ransac_n,
                                                    num_iterations=num_iterations)

    # Extract inlier and outlier point clouds
    inlier_cloud = point_cloud.select_by_index(inliers)
    outlier_cloud = point_cloud.select_by_index(inliers, invert=True)

    # Color the outlier cloud red and the inlier cloud blue
    outlier_cloud.paint_uniform_color([0.8, 0.2, 0.2])  # Red
    inlier_cloud.paint_uniform_color([0.25, 0.5, 0.75])  # Blue
    return outlier_cloud, inlier_cloud

In [3]:
outliers, inliers = ransac(filtred_PCL, distance_threshold=0.1, ransac_n=3, num_iterations=2000)

# Visualize the segmented plane and outliers
o3d.visualization.draw_geometries([inliers,outliers])